In [ ]:
import sys
import xml.dom.minidom

# input file
trace_file = "trace.000.000.000.000.0.1549426071.gBt3uZ.1.xml"

dom = xml.dom.minidom.parse(trace_file)  # or xml.dom.minidom.parseString(xml_string)


In [ ]:
def print_event(event):
    myDict = dict(event.attributes.items())
    print(myDict)

def handle_trace(dom):
    i = 0
    events = dom.getElementsByTagName("Event")
    for e in events:
        print_event(e)
        i += 1
        if i == 5:
            break

handle_trace(dom)

def get_roles(dom):
    events = dom.getElementsByTagName("Event")
    for e in events:
        if e.getAttribute("Type") == "Role":
            print_event(e)
            
get_roles(dom)

In [ ]:
import pandas as pd

# convert dom into a Pandas dataframe
def dom_to_dataframe(dom, columns):
    events = dom.getElementsByTagName("Event")
    data = [dict(e.attributes.items()) for e in events] 
    return pd.DataFrame(data, columns=columns)

columns = ['As', 'ID', 'Locality', 'Machine', 'Severity', 'Transition', 'Time', 'Type']
df = dom_to_dataframe(dom, columns)
print(df.loc[df['Type'] == 'Role'].head(5))

In [ ]:
ss = df.loc[(df['Type'] == 'Role') & (df['As'] == 'StorageServer')]
print(ss)

In [ ]:
from collections import defaultdict

ss = df.loc[(df['Type'] == 'Role') & (df['As'] == 'StorageServer')].astype({'Severity': int, 'Time': float})
endTime =  df[-1:]['Time'].array[0]
print(endTime)
ssMap = {}  # ID to a list of timestamps
nameMap = {}  # markers for this ID
for id in ss.ID.unique():
    # print id
    rows = ss.loc[ss['ID'] == id]
    for index, row in rows.iterrows():
        if id in ssMap:
            ssMap[id].append(row['Time'])
        else:
            ssMap[id] = [row['Time']]
            nameMap[id] = '{}<br>{}'.format(id, row['Machine'])

yMap = {}  # Y axis values for each ID
y = 1
for key, value in ssMap.items():
    if len(value) % 2 is 1:
        value.append(endTime)
    yMap[key] = [y] * len(value)
    y += 1

print(ssMap)
print(yMap)
print(nameMap)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

gdata = []
for key, value in ssMap.items():
    trace = go.Scatter(x=value, y=yMap[key], mode='lines+markers', name=nameMap[key])
    gdata.append(trace)

py.iplot(gdata)